In [4]:
import subprocess
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import csv
from collections import Counter
import numpy as np
from datetime import datetime
import os
import warnings
warnings.filterwarnings('ignore')

# =========== ЭТАП 1: ЗАГРУЗКА ДАННЫХ ===========

def run_tshark_command(command):
    """
    Выполняет команду tshark и возвращает результат
    """
    try:
        result = subprocess.run(command, shell=True, capture_output=True, text=True)
        if result.returncode == 0:
            return result.stdout
        else:
            print(f"Ошибка tshark: {result.stderr}")
            return None
    except Exception as e:
        print(f"Ошибка выполнения команды: {e}")
        return None

def check_tshark_installed():
    """
    Проверяет, установлен ли tshark
    """
    try:
        result = subprocess.run("tshark --version", shell=True, capture_output=True, text=True)
        return result.returncode == 0
    except:
        return False

def extract_dns_from_pcap(pcap_file, packet_limit=None):
    """
    Извлекает DNS-запросы из pcap файла с помощью tshark
    """
    print(f"Извлечение DNS-запросов из {pcap_file}...")
    
    # Формируем команду tshark
    base_cmd = f"tshark -r {pcap_file} -Y dns -T fields"
    fields = [
        "-e frame.time",
        "-e ip.src",
        "-e ip.dst", 
        "-e dns.qry.name",
        "-e dns.qry.type",
        "-e dns.flags.rcode",
        "-e dns.count.queries",
        "-e dns.count.answers"
    ]
    
    if packet_limit:
        base_cmd += f" -c {packet_limit}"
    
    command = f"{base_cmd} {' '.join(fields)} -E header=y"
    
    # Выполняем команду
    output = run_tshark_command(command)
    
    if not output:
        return []
    
    # Парсим вывод
    dns_data = []
    lines = output.strip().split('\n')
    
    if len(lines) < 2:
        print("Нет DNS-запросов в файле")
        return []
    
    # Пропускаем заголовок
    for line in lines[1:]:
        if line.strip():
            parts = line.split('\t')
            if len(parts) >= 8:
                dns_info = {
                    'timestamp': parts[0],
                    'source_ip': parts[1],
                    'dest_ip': parts[2],
                    'query_name': parts[3],
                    'query_type': parts[4],
                    'response_code': parts[5],
                    'query_count': parts[6],
                    'answer_count': parts[7]
                }
                dns_data.append(dns_info)
    
    print(f"Извлечено DNS-запросов: {len(dns_data)}")
    return dns_data

def extract_ip_connections(pcap_file, packet_limit=None):
    """
    Извлекает IP-соединения из pcap файла
    """
    print(f"Извлечение IP-соединений из {pcap_file}...")
    
    base_cmd = f"tshark -r {pcap_file} -Y ip -T fields"
    fields = [
        "-e frame.time",
        "-e ip.src",
        "-e ip.dst",
        "-e ip.proto",
        "-e frame.len",
        "-e tcp.srcport",
        "-e tcp.dstport",
        "-e udp.srcport", 
        "-e udp.dstport",
        "-e _ws.col.Protocol"
    ]
    
    if packet_limit:
        base_cmd += f" -c {packet_limit}"
    
    command = f"{base_cmd} {' '.join(fields)} -E header=y"
    
    output = run_tshark_command(command)
    
    if not output:
        return []
    
    ip_data = []
    lines = output.strip().split('\n')
    
    if len(lines) < 2:
        print("Нет IP-соединений в файле")
        return []
    
    for line in lines[1:]:
        if line.strip():
            parts = line.split('\t')
            if len(parts) >= 10:
                # Определяем порт в зависимости от протокола
                protocol = parts[9] if len(parts) > 9 else "Unknown"
                src_port = parts[5] if parts[5] else (parts[7] if parts[7] else "Unknown")
                dst_port = parts[6] if parts[6] else (parts[8] if parts[8] else "Unknown")
                
                ip_info = {
                    'timestamp': parts[0],
                    'source_ip': parts[1],
                    'dest_ip': parts[2],
                    'ip_proto': parts[3],
                    'frame_len': int(parts[4]) if parts[4].isdigit() else 0,
                    'src_port': src_port,
                    'dst_port': dst_port,
                    'protocol': protocol
                }
                ip_data.append(ip_info)
    
    print(f"Извлечено IP-соединений: {len(ip_data)}")
    return ip_data

def extract_http_requests(pcap_file, packet_limit=None):
    """
    Извлекает HTTP-запросы из pcap файла
    """
    print(f"Извлечение HTTP-запросов из {pcap_file}...")
    
    base_cmd = f"tshark -r {pcap_file} -Y http.request -T fields"
    fields = [
        "-e frame.time",
        "-e ip.src",
        "-e ip.dst",
        "-e http.host",
        "-e http.request.method",
        "-e http.request.uri",
        "-e http.user_agent",
        "-e http.response.code"
    ]
    
    if packet_limit:
        base_cmd += f" -c {packet_limit}"
    
    command = f"{base_cmd} {' '.join(fields)} -E header=y"
    
    output = run_tshark_command(command)
    
    if not output:
        return []
    
    http_data = []
    lines = output.strip().split('\n')
    
    if len(lines) < 2:
        print("Нет HTTP-запросов в файле")
        return []
    
    for line in lines[1:]:
        if line.strip():
            parts = line.split('\t')
            if len(parts) >= 8:
                http_info = {
                    'timestamp': parts[0],
                    'source_ip': parts[1],
                    'dest_ip': parts[2],
                    'host': parts[3],
                    'method': parts[4],
                    'uri': parts[5],
                    'user_agent': parts[6],
                    'response_code': parts[7] if len(parts) > 7 else ""
                }
                http_data.append(http_info)
    
    print(f"Извлечено HTTP-запросов: {len(http_data)}")
    return http_data

def get_packet_statistics(pcap_file):
    """
    Получает общую статистику по pcap файлу
    """
    print(f"Получение статистики по {pcap_file}...")
    
    # Общее количество пакетов
    cmd_count = f"tshark -r {pcap_file} -q -z io,stat,0"
    output = run_tshark_command(cmd_count)
    
    if output:
        lines = output.split('\n')
        for line in lines:
            if "| Number of packets:" in line:
                parts = line.split(':')
                if len(parts) > 1:
                    packet_count = parts[1].strip().split('|')[0].strip()
                    print(f"Общее количество пакетов: {packet_count}")
                    return int(packet_count)
    
    # Если не удалось получить через статистику, используем простой подсчет
    cmd_simple = f"tshark -r {pcap_file} -T fields -e frame.number | tail -1"
    output = run_tshark_command(cmd_simple)
    if output and output.strip().isdigit():
        count = int(output.strip())
        print(f"Общее количество пакетов: {count}")
        return count
    
    print("Не удалось получить статистику пакетов")
    return 0

# =========== ЭТАП 2: АНАЛИЗ И ВЫЯВЛЕНИЕ ПОДОЗРИТЕЛЬНОЙ АКТИВНОСТИ ===========

def analyze_suspicious_patterns(dns_data, ip_data):
    """
    Анализирует данные на наличие подозрительных паттернов
    """
    print("\n=== АНАЛИЗ ПОДОЗРИТЕЛЬНЫХ ПАТТЕРНОВ ===")
    
    suspicious_ips = []
    suspicious_domains = []
    suspicious_patterns = []
    
    # 1. Анализ DNS-запросов
    if dns_data:
        dns_df = pd.DataFrame(dns_data)
        
        # Частые DNS-запросы
        domain_counts = dns_df['query_name'].value_counts()
        frequent_domains = domain_counts[domain_counts > 10].index.tolist()
        
        # Поиск подозрительных доменов
        for domain in dns_df['query_name'].unique():
            if isinstance(domain, str):
                # Длинные домены (возможный DGA)
                if len(domain) > 50:
                    suspicious_domains.append(domain)
                    suspicious_patterns.append(f"Длинное доменное имя: {domain}")
                
                # Домены с цифрами (возможный DGA)
                import re
                if re.search(r'\d{5,}', domain):
                    suspicious_domains.append(domain)
                    suspicious_patterns.append(f"Домен с множеством цифр: {domain}")
                
                # Домены с рандомными последовательностями
                if re.search(r'[a-z]{10,}', domain) and not '.' in domain[10:]:
                    suspicious_domains.append(domain)
                    suspicious_patterns.append(f"Домен с длинной последовательностью букв: {domain}")
                
                # Подозрительные TLD
                suspicious_tlds = ['.xyz', '.top', '.club', '.win', '.bid', '.loan', '.date']
                if any(domain.endswith(tld) for tld in suspicious_tlds):
                    suspicious_domains.append(domain)
                    suspicious_patterns.append(f"Домен с подозрительным TLD: {domain}")
    
    # 2. Анализ IP-адресов
    if ip_data:
        ip_df = pd.DataFrame(ip_data)
        
        # IP-адреса с большим количеством соединений
        src_ip_counts = ip_df['source_ip'].value_counts()
        dest_ip_counts = ip_df['dest_ip'].value_counts()
        
        # Частые источники
        frequent_sources = src_ip_counts[src_ip_counts > 100].index.tolist()
        for ip in frequent_sources:
            suspicious_ips.append(ip)
            suspicious_patterns.append(f"Частый источник трафика: {ip} ({src_ip_counts[ip]} пакетов)")
        
        # Частые получатели
        frequent_destinations = dest_ip_counts[dest_ip_counts > 100].index.tolist()
        for ip in frequent_destinations:
            if ip not in suspicious_ips:
                suspicious_ips.append(ip)
                suspicious_patterns.append(f"Частый получатель трафика: {ip} ({dest_ip_counts[ip]} пакетов)")
        
        # Приватные IP-адреса, общающиеся с внешним миром
        private_ips = []
        for ip in ip_df['source_ip'].unique():
            if ip and (ip.startswith('10.') or ip.startswith('192.168.') or 
                      (ip.startswith('172.') and 16 <= int(ip.split('.')[1]) <= 31)):
                private_ips.append(ip)
        
        # Поиск приватных IP, которые общаются с публичными
        for _, row in ip_df.iterrows():
            src = row['source_ip']
            dst = row['dest_ip']
            if src in private_ips and dst and not (dst.startswith('10.') or dst.startswith('192.168.') or 
                                                  (dst.startswith('172.') and 16 <= int(dst.split('.')[1]) <= 31)):
                suspicious_patterns.append(f"Приватный IP {src} обращается к публичному {dst}")
    
    print(f"Найдено подозрительных IP-адресов: {len(suspicious_ips)}")
    print(f"Найдено подозрительных доменов: {len(suspicious_domains)}")
    
    return suspicious_ips, suspicious_domains, suspicious_patterns

def extract_malicious_indicators(pcap_file):
    """
    Извлекает потенциальные индикаторы компрометации (IOCs)
    """
    print("\n=== ПОИСК ПОТЕНЦИАЛЬНЫХ ИНДИКАТОРОВ КОМПРОМЕТАЦИИ ===")
    
    iocs = {
        'malicious_ips': [],
        'malicious_domains': [],
        'patterns': []
    }
    
    # Известные порты для malware (примеры)
    malicious_ports = {
        4444: "Метасплоит/Бэкдор",
        5555: "Android ADB/Бэкдор",
        6666: "IRC ботнет",
        7777: "Бэкдор",
        8080: "Прокси/VPN",
        8443: "SSL бэкдор",
        8888: "Альтернативный HTTP/Бэкдор",
        9999: "Бэкдор",
        31337: "Элитный порт/Бэкдор"
    }
    
    # Проверка подозрительных портов
    cmd_ports = f"tshark -r {pcap_file} -Y 'tcp.port in {{4444 5555 6666 7777 8080 8443 8888 9999 31337}}' -T fields -e ip.src -e ip.dst -e tcp.port"
    output = run_tshark_command(cmd_ports)
    
    if output:
        lines = output.strip().split('\n')
        for line in lines:
            if line.strip():
                parts = line.split()
                if len(parts) >= 3:
                    src_ip, dst_ip, port = parts[0], parts[1], parts[2]
                    if port in malicious_ports:
                        iocs['patterns'].append(f"Подозрительный порт {port} ({malicious_ports[port]}): {src_ip} -> {dst_ip}")
                        iocs['malicious_ips'].extend([src_ip, dst_ip])
    
    # Поиск известных вредоносных доменов (примеры)
    malicious_keywords = ['malware', 'exploit', 'backdoor', 'botnet', 'c2', 'command', 'control']
    
    cmd_domains = f"tshark -r {pcap_file} -Y dns -T fields -e dns.qry.name"
    output = run_tshark_command(cmd_domains)
    
    if output:
        domains = output.strip().split('\n')
        for domain in domains:
            if any(keyword in domain.lower() for keyword in malicious_keywords):
                iocs['malicious_domains'].append(domain)
                iocs['patterns'].append(f"Домен с подозрительным ключевым словом: {domain}")
    
    print(f"Найдено потенциальных индикаторов: {len(iocs['patterns'])}")
    return iocs

# =========== ЭТАП 3: ВИЗУАЛИЗАЦИЯ РЕЗУЛЬТАТОВ ===========

def create_visualizations(dns_data, ip_data, suspicious_ips, suspicious_domains):
    """
    Создает визуализации результатов анализа
    """
    print("\n=== СОЗДАНИЕ ВИЗУАЛИЗАЦИЙ ===")
    
    # Создаем DataFrame
    dns_df = pd.DataFrame(dns_data) if dns_data else pd.DataFrame()
    ip_df = pd.DataFrame(ip_data) if ip_data else pd.DataFrame()
    
    # Настройка стиля
    plt.style.use('seaborn-v0_8-darkgrid')
    sns.set_palette("husl")
    
    # Создаем фигуру с несколькими графиками
    fig = plt.figure(figsize=(18, 15))
    fig.suptitle('АНАЛИЗ СЕТЕВОГО ТРАФИКА - ОТЧЕТ', fontsize=16, fontweight='bold')
    
    # График 1: DNS запросы по времени
    if not dns_df.empty and 'timestamp' in dns_df.columns:
        ax1 = plt.subplot(3, 3, 1)
        try:
            dns_df['timestamp_dt'] = pd.to_datetime(dns_df['timestamp'])
            dns_df['minute'] = dns_df['timestamp_dt'].dt.floor('Min')
            dns_counts = dns_df.groupby('minute').size()
            
            if not dns_counts.empty:
                ax1.plot(dns_counts.index, dns_counts.values, marker='o', linewidth=2, markersize=4)
                ax1.set_title('DNS-запросы по времени', fontsize=12, fontweight='bold')
                ax1.set_xlabel('Время')
                ax1.set_ylabel('Количество запросов')
                ax1.tick_params(axis='x', rotation=45)
                ax1.grid(True, alpha=0.3)
        except Exception as e:
            print(f"Ошибка при создании графика DNS: {e}")
    
    # График 2: Топ-10 запрашиваемых доменов
    if not dns_df.empty and 'query_name' in dns_df.columns:
        ax2 = plt.subplot(3, 3, 2)
        top_domains = dns_df['query_name'].value_counts().head(10)
        
        if not top_domains.empty:
            y_pos = np.arange(len(top_domains))
            bars2 = ax2.barh(y_pos, top_domains.values)
            ax2.set_yticks(y_pos)
            
            # Укорачиваем длинные домены для отображения
            labels = [d[:30] + '...' if len(d) > 30 else d for d in top_domains.index]
            ax2.set_yticklabels(labels, fontsize=8)
            ax2.set_title('Топ-10 запрашиваемых доменов', fontsize=12, fontweight='bold')
            ax2.set_xlabel('Количество запросов')
            
            for i, (bar, count) in enumerate(zip(bars2, top_domains.values)):
                ax2.text(count, bar.get_y() + bar.get_height()/2, 
                        f' {count}', va='center', fontsize=8)
    
    # График 3: Распределение протоколов
    if not ip_df.empty and 'protocol' in ip_df.columns:
        ax3 = plt.subplot(3, 3, 3)
        protocol_counts = ip_df['protocol'].value_counts().head(8)
        
        if not protocol_counts.empty:
            wedges, texts, autotexts = ax3.pie(protocol_counts.values, labels=protocol_counts.index,
                                               autopct='%1.1f%%', startangle=90)
            ax3.set_title('Распределение протоколов', fontsize=12, fontweight='bold')
            for autotext in autotexts:
                autotext.set_color('white')
                autotext.set_fontweight('bold')
    
    # График 4: Топ-10 IP-адресов источников
    if not ip_df.empty and 'source_ip' in ip_df.columns:
        ax4 = plt.subplot(3, 3, 4)
        top_sources = ip_df['source_ip'].value_counts().head(10)
        
        if not top_sources.empty:
            x_pos = np.arange(len(top_sources))
            bars4 = ax4.bar(x_pos, top_sources.values)
            ax4.set_title('Топ-10 IP-адресов источников', fontsize=12, fontweight='bold')
            ax4.set_ylabel('Количество пакетов')
            ax4.set_xticks(x_pos)
            
            # Укорачиваем IP для отображения
            labels = [ip[:15] + '...' if len(ip) > 15 else ip for ip in top_sources.index]
            ax4.set_xticklabels(labels, rotation=45, ha='right', fontsize=8)
            
            for bar, count in zip(bars4, top_sources.values):
                height = bar.get_height()
                ax4.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                        f'{count}', ha='center', va='bottom', fontsize=8)
    
    # График 5: Топ-10 IP-адресов получателей
    if not ip_df.empty and 'dest_ip' in ip_df.columns:
        ax5 = plt.subplot(3, 3, 5)
        top_dests = ip_df['dest_ip'].value_counts().head(10)
        
        if not top_dests.empty:
            x_pos = np.arange(len(top_dests))
            bars5 = ax5.bar(x_pos, top_dests.values)
            ax5.set_title('Топ-10 IP-адресов получателей', fontsize=12, fontweight='bold')
            ax5.set_ylabel('Количество пакетов')
            ax5.set_xticks(x_pos)
            
            labels = [ip[:15] + '...' if len(ip) > 15 else ip for ip in top_dests.index]
            ax5.set_xticklabels(labels, rotation=45, ha='right', fontsize=8)
            
            for bar, count in zip(bars5, top_dests.values):
                height = bar.get_height()
                ax5.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                        f'{count}', ha='center', va='bottom', fontsize=8)
    
    # График 6: Размеры пакетов
    if not ip_df.empty and 'frame_len' in ip_df.columns:
        ax6 = plt.subplot(3, 3, 6)
        # Фильтруем выбросы
        lengths = ip_df['frame_len'].astype(float)
        q1 = lengths.quantile(0.25)
        q3 = lengths.quantile(0.75)
        iqr = q3 - q1
        filtered_lengths = lengths[(lengths >= q1 - 1.5*iqr) & (lengths <= q3 + 1.5*iqr)]
        
        ax6.hist(filtered_lengths, bins=50, alpha=0.7, edgecolor='black')
        ax6.set_title('Распределение размеров пакетов', fontsize=12, fontweight='bold')
        ax6.set_xlabel('Размер пакета (байты)')
        ax6.set_ylabel('Количество пакетов')
        ax6.grid(True, alpha=0.3)
    
    # График 7: Сетевой трафик по времени
    if not ip_df.empty and 'timestamp' in ip_df.columns:
        ax7 = plt.subplot(3, 3, 7)
        try:
            ip_df['timestamp_dt'] = pd.to_datetime(ip_df['timestamp'])
            ip_df['minute'] = ip_df['timestamp_dt'].dt.floor('Min')
            traffic_counts = ip_df.groupby('minute').size()
            
            if not traffic_counts.empty:
                ax7.plot(traffic_counts.index, traffic_counts.values, marker='.', 
                        linewidth=1.5, color='red', alpha=0.7)
                ax7.set_title('Сетевой трафик по времени', fontsize=12, fontweight='bold')
                ax7.set_xlabel('Время')
                ax7.set_ylabel('Пакетов в минуту')
                ax7.tick_params(axis='x', rotation=45)
                ax7.grid(True, alpha=0.3)
        except Exception as e:
            print(f"Ошибка при создании графика трафика: {e}")
    
    # График 8: Подозрительные активности
    ax8 = plt.subplot(3, 3, 8)
    categories = ['Подозрительные IP', 'Подозрительные домены']
    values = [len(suspicious_ips), len(suspicious_domains)]
    
    bars8 = ax8.bar(categories, values, color=['#ff6b6b', '#ffa726'])
    ax8.set_title('Подозрительная активность', fontsize=12, fontweight='bold')
    ax8.set_ylabel('Количество')
    
    for bar, value in zip(bars8, values):
        height = bar.get_height()
        ax8.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                f'{value}', ha='center', va='bottom', fontsize=10)
    
    # График 9: Типы DNS-запросов
    if not dns_df.empty and 'query_type' in dns_df.columns:
        ax9 = plt.subplot(3, 3, 9)
        query_types = dns_df['query_type'].value_counts().head(10)
        
        if not query_types.empty:
            # Маппинг числовых типов к названиям
            type_names = {
                '1': 'A',
                '28': 'AAAA',
                '5': 'CNAME',
                '15': 'MX',
                '2': 'NS',
                '16': 'TXT',
                '6': 'SOA',
                '12': 'PTR'
            }
            
            labels = [type_names.get(str(t), str(t)) for t in query_types.index]
            bars9 = ax9.bar(range(len(query_types)), query_types.values)
            ax9.set_title('Типы DNS-запросов', fontsize=12, fontweight='bold')
            ax9.set_ylabel('Количество запросов')
            ax9.set_xticks(range(len(query_types)))
            ax9.set_xticklabels(labels, rotation=45, ha='right', fontsize=8)
            
            for bar, count in zip(bars9, query_types.values):
                height = bar.get_height()
                ax9.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                        f'{count}', ha='center', va='bottom', fontsize=8)
    
    plt.tight_layout()
    plt.savefig('network_analysis_report.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("✓ Визуализации сохранены в 'network_analysis_report.png'")

def save_results_to_files(dns_data, ip_data, suspicious_ips, suspicious_domains, suspicious_patterns):
    """
    Сохраняет результаты анализа в файлы
    """
    print("\n=== СОХРАНЕНИЕ РЕЗУЛЬТАТОВ ===")
    
    # Сохраняем DNS-запросы в CSV
    if dns_data:
        dns_df = pd.DataFrame(dns_data)
        dns_df.to_csv('dns_requests.csv', index=False, encoding='utf-8')
        print(f"✓ DNS-запросы сохранены в 'dns_requests.csv' ({len(dns_df)} записей)")
    
    # Сохраняем IP-соединения в CSV
    if ip_data:
        ip_df = pd.DataFrame(ip_data)
        ip_df.to_csv('ip_connections.csv', index=False, encoding='utf-8')
        print(f"✓ IP-соединения сохранены в 'ip_connections.csv' ({len(ip_df)} записей)")
    
    # Сохраняем подозрительные находки
    with open('suspicious_findings.txt', 'w', encoding='utf-8') as f:
        f.write("="*80 + "\n")
        f.write("ОТЧЕТ О ПОДОЗРИТЕЛЬНОЙ СЕТЕВОЙ АКТИВНОСТИ\n")
        f.write("="*80 + "\n\n")
        
        f.write("СГЕНЕРИРОВАНО: " + datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n")
        f.write("="*80 + "\n\n")
        
        f.write("1. ПОДОЗРИТЕЛЬНЫЕ IP-АДРЕСА\n")
        f.write("-" * 40 + "\n")
        if suspicious_ips:
            for ip in suspicious_ips:
                f.write(f"  • {ip}\n")
        else:
            f.write("  Не обнаружено\n")
        
        f.write("\n2. ПОДОЗРИТЕЛЬНЫЕ ДОМЕННЫЕ ИМЕНА\n")
        f.write("-" * 40 + "\n")
        if suspicious_domains:
            for domain in suspicious_domains[:50]:  # Ограничиваем вывод
                f.write(f"  • {domain}\n")
        else:
            f.write("  Не обнаружено\n")
        
        f.write("\n3. ОБНАРУЖЕННЫЕ ПАТТЕРНЫ УГРОЗ\n")
        f.write("-" * 40 + "\n")
        if suspicious_patterns:
            for i, pattern in enumerate(suspicious_patterns[:50], 1):  # Ограничиваем вывод
                f.write(f"  {i:2d}. {pattern}\n")
        else:
            f.write("  Не обнаружено\n")
        
        f.write("\n" + "="*80 + "\n")
        f.write("РЕКОМЕНДАЦИИ ПО БЕЗОПАСНОСТИ:\n")
        f.write("-" * 40 + "\n")
        f.write("1. Проверьте подозрительные IP-адреса на blacklists\n")
        f.write("2. Исследуйте подозрительные домены на предмет вредоносности\n")
        f.write("3. Настройте правила файервола для блокировки подозрительных адресов\n")
        f.write("4. Установите мониторинг для обнаружения подобных паттернов\n")
        f.write("5. Проведите проверку систем на наличие компрометации\n")
    
    print("✓ Подозрительные находки сохранены в 'suspicious_findings.txt'")
    
    # Сохраняем краткий отчет в JSON
    report = {
        'timestamp': datetime.now().isoformat(),
        'suspicious_ips_count': len(suspicious_ips),
        'suspicious_domains_count': len(suspicious_domains),
        'suspicious_patterns_count': len(suspicious_patterns),
        'top_suspicious_ips': suspicious_ips[:10],
        'top_suspicious_domains': suspicious_domains[:10],
        'analysis_summary': {
            'total_dns_queries': len(dns_data) if dns_data else 0,
            'total_ip_connections': len(ip_data) if ip_data else 0,
            'suspicious_activity_detected': len(suspicious_ips) > 0 or len(suspicious_domains) > 0
        }
    }
    
    with open('analysis_report.json', 'w', encoding='utf-8') as f:
        json.dump(report, f, indent=2, ensure_ascii=False)
    
    print("✓ Краткий отчет сохранен в 'analysis_report.json'")

def generate_command_line_report(dns_data, ip_data, suspicious_ips, suspicious_domains, iocs):
    """
    Генерирует отчет для вывода в командной строке
    """
    print("\n" + "="*80)
    print("ОТЧЕТ ПО АНАЛИЗУ СЕТЕВОГО ТРАФИКА")
    print("="*80)
    
    # Общая статистика
    print(f"\n📊 ОБЩАЯ СТАТИСТИКА:")
    print(f"   • DNS-запросы: {len(dns_data)}")
    print(f"   • IP-соединения: {len(ip_data)}")
    
    if ip_data:
        ip_df = pd.DataFrame(ip_data)
        unique_src = ip_df['source_ip'].nunique()
        unique_dst = ip_df['dest_ip'].nunique()
        print(f"   • Уникальных IP-источников: {unique_src}")
        print(f"   • Уникальных IP-получателей: {unique_dst}")
    
    # Подозрительная активность
    print(f"\n⚠️  ПОДОЗРИТЕЛЬНАЯ АКТИВНОСТЬ:")
    print(f"   • Подозрительных IP-адресов: {len(suspicious_ips)}")
    print(f"   • Подозрительных доменов: {len(suspicious_domains)}")
    print(f"   • Индикаторов компрометации: {len(iocs['patterns'])}")
    
    # Топ подозрительных IP
    if suspicious_ips:
        print(f"\n🔴 ТОП-5 ПОДОЗРИТЕЛЬНЫХ IP-АДРЕСОВ:")
        for i, ip in enumerate(suspicious_ips[:5], 1):
            print(f"   {i}. {ip}")
    
    # Топ подозрительных доменов
    if suspicious_domains:
        print(f"\n🔴 ТОП-5 ПОДОЗРИТЕЛЬНЫХ ДОМЕНОВ:")
        for i, domain in enumerate(suspicious_domains[:5], 1):
            print(f"   {i}. {domain}")
    
    # Индикаторы компрометации
    if iocs['patterns']:
        print(f"\n🚨 ОБНАРУЖЕННЫЕ ИНДИКАТОРЫ КОМПРОМЕТАЦИИ:")
        for i, pattern in enumerate(iocs['patterns'][:5], 1):
            print(f"   {i}. {pattern}")
    
    # Рекомендации
    print(f"\n💡 РЕКОМЕНДАЦИИ:")
    if suspicious_ips or suspicious_domains or iocs['patterns']:
        print("   1. НЕМЕДЛЕННО ИССЛЕДОВАТЬ указанные IP и домены")
        print("   2. Проверить системы на наличие вредоносного ПО")
        print("   3. Настроить правила файервола для блокировки подозрительных адресов")
        print("   4. Усилить мониторинг сетевой активности")
        print("   5. Обновить антивирусные базы и провести полное сканирование")
    else:
        print("   ✅ Явных признаков компрометации не обнаружено")
        print("   Рекомендуется продолжить регулярный мониторинг")
    
    print("\n" + "="*80)
    print("📁 СОЗДАННЫЕ ФАЙЛЫ:")
    print("   • network_analysis_report.png - Визуальный отчет")
    print("   • dns_requests.csv - DNS-запросы")
    print("   • ip_connections.csv - IP-соединения")
    print("   • suspicious_findings.txt - Подозрительные находки")
    print("   • analysis_report.json - Краткий отчет в JSON")
    print("="*80)

# =========== ОСНОВНАЯ ФУНКЦИЯ ===========

def main():
    """
    Основная функция для анализа сетевого дампа
    """
    print("="*80)
    print("АНАЛИЗАТОР СЕТЕВЫХ ДАМПОВ (TSHARK VERSION)")
    print("="*80)
    
    # Проверяем, установлен ли tshark
    if not check_tshark_installed():
        print("❌ ОШИБКА: tshark не установлен или не найден в PATH")
        print("\nУстановите Wireshark, который включает tshark:")
        print("  Windows: https://www.wireshark.org/download.html")
        print("  Linux: sudo apt-get install wireshark-tshark")
        print("  Mac: brew install wireshark")
        return
    
    print("✓ tshark обнаружен")
    
    # Укажите путь к вашему файлу pcapng
    pcap_file = "traffic.pcapng"  # Замените на путь к вашему файлу
    
    # Проверяем существование файла
    if not os.path.exists(pcap_file):
        print(f"\n❌ Файл {pcap_file} не найден!")
        print("\nПожалуйста:")
        print("  1. Поместите ваш pcapng файл в текущую папку")
        print("  2. Переименуйте его в 'traffic.pcapng'")
        print("  3. Или укажите полный путь в переменной pcap_file")
        print("\nПримеры тестовых файлов можно скачать:")
        print("  • https://wiki.wireshark.org/SampleCaptures")
        print("  • https://www.malware-traffic-analysis.net/")
        return
    
    print(f"✓ Файл {pcap_file} найден")
    
    # Ограничение количества пакетов для анализа (None для всех)
    packet_limit = 10000  # Для быстрого анализа
    
    # ЭТАП 1: Извлечение данных
    print("\n" + "="*80)
    print("ЭТАП 1: ИЗВЛЕЧЕНИЕ ДАННЫХ ИЗ PCAP")
    print("="*80)
    
    dns_data = extract_dns_from_pcap(pcap_file, packet_limit)
    ip_data = extract_ip_connections(pcap_file, packet_limit)
    http_data = extract_http_requests(pcap_file, packet_limit)
    
    if not dns_data and not ip_data:
        print("\n❌ Не удалось извлечь данные из файла")
        print("Проверьте, что файл содержит сетевой трафик")
        return
    
    # ЭТАП 2: Анализ подозрительной активности
    print("\n" + "="*80)
    print("ЭТАП 2: АНАЛИЗ ПОДОЗРИТЕЛЬНОЙ АКТИВНОСТИ")
    print("="*80)
    
    suspicious_ips, suspicious_domains, suspicious_patterns = analyze_suspicious_patterns(dns_data, ip_data)
    
    # Поиск индикаторов компрометации
    iocs = extract_malicious_indicators(pcap_file)
    
    # ЭТАП 3: Визуализация и сохранение результатов
    print("\n" + "="*80)
    print("ЭТАП 3: ВИЗУАЛИЗАЦИЯ И СОХРАНЕНИЕ РЕЗУЛЬТАТОВ")
    print("="*80)
    
    create_visualizations(dns_data, ip_data, suspicious_ips, suspicious_domains)
    save_results_to_files(dns_data, ip_data, suspicious_ips, suspicious_domains, suspicious_patterns)
    
    # Генерация отчета
    generate_command_line_report(dns_data, ip_data, suspicious_ips, suspicious_domains, iocs)
    
    print("\n" + "="*80)
    print("✅ АНАЛИЗ ЗАВЕРШЕН УСПЕШНО!")
    print("="*80)

# Запуск анализа
if __name__ == "__main__":
    main()

Exception in thread Thread-6 (_readerthread):
Traceback (most recent call last):
  File "c:\Users\Selecty\anaconda3\Lib\threading.py", line 1043, in _bootstrap_inner
    self.run()
    ~~~~~~~~^^
  File "c:\Users\Selecty\anaconda3\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
    ~~~~~~~~~~~~~~~~~~~~~^^^^^^
  File "c:\Users\Selecty\anaconda3\Lib\threading.py", line 994, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Selecty\anaconda3\Lib\subprocess.py", line 1615, in _readerthread
    buffer.append(fh.read())
                  ~~~~~~~^^
  File "<frozen codecs>", line 325, in decode
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xad in position 9: invalid start byte


АНАЛИЗАТОР СЕТЕВЫХ ДАМПОВ (TSHARK VERSION)
❌ ОШИБКА: tshark не установлен или не найден в PATH

Установите Wireshark, который включает tshark:
  Windows: https://www.wireshark.org/download.html
  Linux: sudo apt-get install wireshark-tshark
  Mac: brew install wireshark


Я не знаю как еще визуализировать, так что формально нашел что требовалось.